# AI Sem2 2019 Tutorial 2 Soltuions

Alex | 31 July 2019

In [1]:
import numpy as np
import time

# consider converting all data to np.uint8 for speed

# BFS

In [2]:
class Node():
    def __init__(self, state, parent=None, action=None, depth=0):
        self.state = state  # a board
        self.parent = parent  # parent node, a NODE! not just a matrix.
        self.action = action  # The one that led to this node (useful for retracing purpose)
        self.depth = depth  # depth of the node in the tree. This is the criterion for who's next in DFS, BFS.
    
    def world_dynamics(self, current_state, action):
        # Firstly, find out where the white tile is
        row, col = np.where(current_state == 0)
        new_state = current_state.copy()
        if action == 'up':
            if row - 1 < 0:  # It is valid action only if new index is still within the matrix
                raise Exception('Inappropriate action for the current state')
            new_state[row, col], new_state[row-1, col] = new_state[row-1, col], new_state[row, col]
        elif action == 'down':
            if row + 1 > 2:  # It is valid action only if new index is still within the matrix
                raise Exception('Inappropriate action for the current state')
            new_state[row, col], new_state[row+1, col] = new_state[row+1, col], new_state[row, col]        
        elif action == 'left':
            if col - 1 < 0:  # It is valid action only if new index is still within the matrix
                raise Exception('Inappropriate action for the current state')
            new_state[row, col], new_state[row, col-1] = new_state[row, col-1], new_state[row, col]
        elif action == 'right':
            if col + 1 > 2:  # It is valid action only if new index is still within the matrix
                raise Exception('Inappropriate action for the current state')
            new_state[row, col], new_state[row, col+1] = new_state[row, col+1], new_state[row, col]
        else: print('Unknown action!')
        return new_state
    
    def explore_actions(self, state):
        possibilities = ['up', 'down', 'right', 'left']
        actions = []
        for apossibility in possibilities:
            try: 
                new_state = self.world_dynamics(state, apossibility)
                actions.append(apossibility)  # if world_dynamics didn't return False
            except: pass  # move on to the next possibility
        return actions
    
    def whos_next_BFS(self, frontier):
        """
        This is an ideal function. It relies on .depth attributes of nodes.
        And determines which node should be explored first according to BFS regime.
        However, there's a faster implementation of it. Which is FIFO frontier.
        This list by construction will make the most recent nodes (which are the deepest) be explored later
        While older nodes in the list go first. So, no need for this function. It is only here for perfection.
        Pythonically speaking, use .pop(0) and .append methods of lists to build a FIFO type queue
        """
        least_depth = 99999999999
        least_depth_node_index = None
        for index, anode in enumerate(frontier):
            if anode.depth < least_depth:
                least_depth = anode.depth
                least_depth_node_index = index
        return least_depth_node_index

    def done(self, current_node):
        """ The prupose of this function  is: Trace back this node to the founding granpa.
        Print out the states through out
        """
        founding_father = current_node
        states = []  # the retraced states will be stored here.
        counter = 0
        limit = 50  # if the trace is longer than 50, don't print anything, it will be a mess.
        while founding_father:
            states.append(founding_father.state)
            founding_father = founding_father.parent
            counter += 1
            # Keep doing this until you reach the founding father that has a parent None (see default of init method)
        print('Number of steps to the goal = ', counter)
        if counter > limit:
            print('Too many steps to be printed')
        else:
            for i in reversed(states):  # Cause we want to print solution from initial to goal not the opposite.
                print(i, '\n')

    def BFS(self, goal_state):
        start = time.time()
        frontier = [self]  # queue of found but unvisited nodes, FIFO
        frontier_max_size = len(frontier)  # We measure space / memory requirements of the algo via this metric.
        ft = set()
        ft.add(tuple(self.state.reshape(9,)))
        # A version of the frontier that contains only states, not nodes
        # This is version is required because Python is slow, and having this numeric version
        # will make things much faster.
        
        explored = set([])  # We use this to measure time requirements (#visited_nodes). This is unbiased measure and doesn't depend on computer speed
        # We should have another set of unexplored, but it is huge and we will ignore it.
        # The union of the three sets above is the state space.

        
        # Let's start exploring the frontier.
        ct = 0  # a counter to see the progress.
        while frontier:
            '''
            As long as there are elements in the frontier, then the search is on. But, this can be an infinite loop
            in case of graph, unless, we store the visited cases! 
            The only way you can terminate is via return of BFS function
            that will interrupt the while loop. So, the function spits out the first solution it finds.
            '''
            ct += 1
            print(ct, end='\r')
            
            if len(frontier) > frontier_max_size: frontier_max_size = len(frontier)
            # This is a measure of memory requirements. Although storing explored elements kills
            # difference between DFS and BFS cause DFS is promoted as having small memory requirements, but when visited nodes
            # are stored in addition to frontier, at some point, there will be no big difference.
            # This is a cost we pay to convert a graph to a tree.
            current_node = frontier.pop(0) # select and remove the first node in the queue
            ft.remove(tuple(current_node.state.reshape(9,)))
            explored.add(tuple(current_node.state.reshape(9,)))  # cause we are going to explore it.
            # The reason why it is stored as a tuple, is to make elements hashable, so then we can ask if an element is in
            # the list
            
            # Firstly, let's check if the new node is the goal:
            if np.array_equal(current_node.state, goal_state):
                print('Time required = ', -start + time.time())
                print('Explored states = ', len(explored))
                print('Frontier max size = ', frontier_max_size)
                self.done(current_node)
                '''
                Time required shows overall performance. This could be different indicator from explored states figure
                Because there can be a costy h function that leads to less explored states but more computation time.
                So, there is no point of it. Additionally, run time can depend on computer specs, and representation
                of states (a matrix or a string)
                Frontier max size is indication of storage requirements.
                Number of steps to reach the solution is indication of how optimal the solution  is.
                '''
                return True  # This return is for BFS method. It is a mean to break out of the while loop.
            
            # if it is not the goal, then, let's dig deeper:
            actions = self.explore_actions(current_node.state)  # branches that can come out
            for anaction in actions:  # add exploration results to the frontier.
                new_state = self.world_dynamics(current_node.state, anaction)
                if tuple(new_state.reshape(9,)) not in explored:
                    """ cause we are imposing a tree search on a graph problem
                    The problem has cycles, something that doesn't exist in tree searches. So, we have to
                    prevent that.
                    A tree is like exploring your folders in HDD. you cannot open a subfolder and it takes you
                    to an earlier folder. In graphs this is possible. An example of a graph is a map.
                    """
                    if tuple(new_state.reshape(9,)) not in ft:
                        new_node = Node(state=new_state, parent=current_node, action=anaction, depth=current_node.depth+1)
                        frontier.append(new_node)
                        ft.add(tuple(new_state.reshape(9,)))
                        
        print('Failed to reach target goal. Number of states explored = ')
        return len(explored)  # i.e. frontier list was emptied, all state space was explored, goal wasn't reached.
        # The result returned above should be equal to half of the state space size since it was proven that the other 
        # half is unsolvable. The state space cardinality is 9! half of them has parity "odd" and half got parity "even"
        # Meaning half of them fall into one subset and so is the other half. In each subset you can move freely 
        # between any two states. Additionally, when starting in one subset, you're stuck in it, and thus, there are 
        # only 9! / 2 states to explore.

In [3]:
def get_inv_count(array):
    """
    This function takes in a state, and returns the inversion number for it.
    """
    counter = 0
    linear = array.reshape(-1,).tolist()
    linear.remove(0)  # remove the empty tile
    N = len(linear)
    for index, item in enumerate(linear):
        for i in range(index+1, N, 1):
            if linear[i] < item:
                counter += 1
    return counter


def check_solvability(array1, array2):
    """
    This function tells whether you can solve the puzzle starting from array1 and your goal is array2.
    It turned out that one thing is invariant under blank tile slidings: parity of total number of inversions.
    If it is the same for array1 and array2, then, there's a sequence of slidings to move from one to the other.
    An inversion is defined as how many numbers (tiles), a certain number has preceeded, i.e. it came before them, even though
    they're smaller (broke the ascening order). You take a number, and you start looking at all the number that
    came after it, if you ever observe anything larger than the number you're considering, that's +1 inversion.
    PS: blank tile is ignored, and the board is thought of in a linear row major order.
    """
    count1, count2 = get_inv_count(array1), get_inv_count(array2)
    parity1 = 'odd' if (count1 % 2 == 1) else 'even'
    parity2 = 'odd' if (count2 % 2 == 1) else 'even'
    return parity1 == parity2

# Easy problem to check

In [4]:
test = np.array([1,2,3,8,6,4,7,5,0], dtype=np.uint8).reshape(3,3)
goal_state = np.array([1,2,3,8,0,4,7,6,5], dtype=np.uint8).reshape(3,3)
initial_state = test
print(initial_state, '\n')
print(goal_state, '\n')

root_node = Node(state=initial_state)

[[1 2 3]
 [8 6 4]
 [7 5 0]] 

[[1 2 3]
 [8 0 4]
 [7 6 5]] 



In [5]:
root_node.world_dynamics(initial_state, 'up')

array([[1, 2, 3],
       [8, 6, 0],
       [7, 5, 4]], dtype=uint8)

In [6]:
root_node.BFS(goal_state)

Time required =  0.0049839019775390625
Explored states =  6
Frontier max size =  6
Number of steps to the goal =  3
[[1 2 3]
 [8 6 4]
 [7 5 0]] 

[[1 2 3]
 [8 6 4]
 [7 0 5]] 

[[1 2 3]
 [8 0 4]
 [7 6 5]] 



True

# Hard random, probably unsolvable goal

In [7]:
# Run this cell till you get a solvable/ unsolvable config, whichever you want
goal_state = np.arange(9, dtype=np.uint8)
np.random.shuffle(goal_state)
goal_state = goal_state.reshape(3,3)
print(goal_state)
print(check_solvability(goal_state, initial_state))

[[4 3 6]
 [8 7 5]
 [1 0 2]]
True


In [8]:

root_node.BFS(goal_state)


Time required =  41.22171950340271
Explored states =  101542
Frontier max size =  23972
Number of steps to the goal =  24
[[1 2 3]
 [8 6 4]
 [7 5 0]] 

[[1 2 3]
 [8 6 0]
 [7 5 4]] 

[[1 2 3]
 [8 0 6]
 [7 5 4]] 

[[1 0 3]
 [8 2 6]
 [7 5 4]] 

[[1 3 0]
 [8 2 6]
 [7 5 4]] 

[[1 3 6]
 [8 2 0]
 [7 5 4]] 

[[1 3 6]
 [8 2 4]
 [7 5 0]] 

[[1 3 6]
 [8 2 4]
 [7 0 5]] 

[[1 3 6]
 [8 0 4]
 [7 2 5]] 

[[1 3 6]
 [8 4 0]
 [7 2 5]] 

[[1 3 0]
 [8 4 6]
 [7 2 5]] 

[[1 0 3]
 [8 4 6]
 [7 2 5]] 

[[1 4 3]
 [8 0 6]
 [7 2 5]] 

[[1 4 3]
 [0 8 6]
 [7 2 5]] 

[[0 4 3]
 [1 8 6]
 [7 2 5]] 

[[4 0 3]
 [1 8 6]
 [7 2 5]] 

[[4 3 0]
 [1 8 6]
 [7 2 5]] 

[[4 3 6]
 [1 8 0]
 [7 2 5]] 

[[4 3 6]
 [1 8 5]
 [7 2 0]] 

[[4 3 6]
 [1 8 5]
 [7 0 2]] 

[[4 3 6]
 [1 8 5]
 [0 7 2]] 

[[4 3 6]
 [0 8 5]
 [1 7 2]] 

[[4 3 6]
 [8 0 5]
 [1 7 2]] 

[[4 3 6]
 [8 7 5]
 [1 0 2]] 



True

# Starting and Goal states required in Tutorial sheet

In [9]:
s1 = np.array([1, 3, 4, 8, 6, 2, 7, 0, 5]).reshape(3,3)
g1 = np.array([1, 2, 3, 8, 0, 4, 7, 6, 5]).reshape(3,3)

s2 = np.array([2, 8, 1, 0, 4, 3, 7, 6, 5]).reshape(3,3)
g2 = np.array([1, 2, 3, 8, 0, 4, 7, 6, 5]).reshape(3,3)

s3 = np.array([2, 8, 1, 4, 6, 3, 0, 7, 5]).reshape(3,3)
g3 = np.array([1, 2, 3, 8, 0, 4, 7, 6, 5]).reshape(3,3)



In [10]:
root = Node(s1)
root.BFS(g1)

Time required =  0.01898360252380371
Explored states =  38
Frontier max size =  32
Number of steps to the goal =  6
[[1 3 4]
 [8 6 2]
 [7 0 5]] 

[[1 3 4]
 [8 0 2]
 [7 6 5]] 

[[1 3 4]
 [8 2 0]
 [7 6 5]] 

[[1 3 0]
 [8 2 4]
 [7 6 5]] 

[[1 0 3]
 [8 2 4]
 [7 6 5]] 

[[1 2 3]
 [8 0 4]
 [7 6 5]] 



True

# DFS

In [11]:
# Now let us implement DFS
class Node1(Node):
    def __init__(self, **kwargs):
        super(Node1, self).__init__(**kwargs)

    def DFS(self, goal_state):
        start = time.time()
        frontier = [self]  # queue of found but unvisited nodes, FILO
        ft = set()
        ft.add(tuple(self.state.reshape(9,)))
        # A version of the frontier that contains only states, not nodes
        # This is version is required because Python is slow, and having this numeric version
        # will make things much faster.
        frontier_max_size = len(frontier)
        explored = set([])
        ct = 0
        while frontier:
            ct += 1
            print(ct, end='\r')
            if len(frontier) > frontier_max_size: frontier_max_size = len(frontier)
            current_node = frontier.pop()  # FILO            
            ft.remove(tuple(current_node.state.reshape(9,)))
            explored.add(tuple(current_node.state.reshape(9,)))  # cause we are going to explore it.

            if np.array_equal(current_node.state, goal_state):
                print('Time required = ', -start + time.time())
                print('Explored states = ', len(explored))
                print('Frontier max size = ', frontier_max_size)
                self.done(current_node)
                return True  # This return is for BFS method. It is a mean to break out of the while loop.

            actions = self.explore_actions(current_node.state)  # branches that can come out
            for anaction in actions:  # add exploration results to the frontier.
                new_state = self.world_dynamics(current_node.state, anaction)
                if tuple(new_state.reshape(9,)) not in explored:
                    if tuple(new_state.reshape(9,)) not in ft:
                        new_node = Node1(state=new_state, parent=current_node, action=anaction, depth=current_node.depth+1)
                        frontier.append(new_node)
                        ft.add(tuple(new_state.reshape(9,)))
        print('Failed to reach target goal. Number of states explored = ')
        return len(explored)

In [12]:
root_node1 = Node1(state=initial_state)

In [13]:
root_node1.DFS(goal_state)

Time required =  20.976402521133423
Explored states =  52624
Frontier max size =  33372
Number of steps to the goal =  46982
Too many steps to be printed


True

# Uinform cost search

In [14]:
class Node2(Node1):  # this way, we will inheret DFS and BFS methods
    def __init__(self, path_cost=0, **kwargs):
        super(Node2, self).__init__(**kwargs)
        self.path_cost = path_cost

    def whos_next_UCS(self, frontier):  
        return np.argmin([anelement.path_cost for anelement in frontier])
    
    def UCS(self, goal_state):
        start = time.time()
        frontier = [self] 
        frontier_max_size = len(frontier)
        explored = set([])
        
        while frontier:
            if len(frontier) > frontier_max_size: frontier_max_size = len(frontier)
            index = self.whos_next_UCS(frontier)
            current_node = frontier[index] # select, then remove the first node in the queue
            del frontier[index]
            print(len(frontier), end='\r')
            
            explored.add(tuple(current_node.state.reshape(9,)))  # cause we are going to explore it.

            if np.array_equal(current_node.state, goal_state):
                print('Time required = ', -start + time.time())
                print('Explored states = ', len(explored))
                print('Frontier max size = ', frontier_max_size)
                print('Cost of solution path = ', current_node.path_cost)
                self.done(current_node)
                return True  # This return is for BFS method. It is a mean to break out of the while loop.
            
            actions = self.explore_actions(current_node.state)  # branches that can come out
            for anaction in actions:  # add exploration results to the frontier.
                new_state = self.world_dynamics(current_node.state, anaction)
                if tuple(new_state.reshape(9,)) not in explored:  
                    # Consider to add another check for child not being in frontier.
                    new_node = Node2(state=new_state, parent=current_node, action=anaction, depth=current_node.depth+1,
                                   path_cost=current_node.path_cost+cost_dict[anaction])
                    frontier.append(new_node)
        print('Failed to reach target goal. Number of states explored = ')
        return len(explored)

In [15]:
cost_dict = {'up':1, 'down':2, 'left':3, 'right':4}

In [16]:
root_node3 = Node2(state=initial_state)

In [17]:
root_node3.UCS(goal_state)

Time required =  986.5350604057312
Explored states =  108250
Frontier max size =  54919
Cost of solution path =  56
Number of steps to the goal =  24
[[1 2 3]
 [8 6 4]
 [7 5 0]] 

[[1 2 3]
 [8 6 0]
 [7 5 4]] 

[[1 2 3]
 [8 0 6]
 [7 5 4]] 

[[1 0 3]
 [8 2 6]
 [7 5 4]] 

[[1 3 0]
 [8 2 6]
 [7 5 4]] 

[[1 3 6]
 [8 2 0]
 [7 5 4]] 

[[1 3 6]
 [8 2 4]
 [7 5 0]] 

[[1 3 6]
 [8 2 4]
 [7 0 5]] 

[[1 3 6]
 [8 0 4]
 [7 2 5]] 

[[1 3 6]
 [0 8 4]
 [7 2 5]] 

[[0 3 6]
 [1 8 4]
 [7 2 5]] 

[[3 0 6]
 [1 8 4]
 [7 2 5]] 

[[3 8 6]
 [1 0 4]
 [7 2 5]] 

[[3 8 6]
 [1 4 0]
 [7 2 5]] 

[[3 8 6]
 [1 4 5]
 [7 2 0]] 

[[3 8 6]
 [1 4 5]
 [7 0 2]] 

[[3 8 6]
 [1 4 5]
 [0 7 2]] 

[[3 8 6]
 [0 4 5]
 [1 7 2]] 

[[3 8 6]
 [4 0 5]
 [1 7 2]] 

[[3 0 6]
 [4 8 5]
 [1 7 2]] 

[[0 3 6]
 [4 8 5]
 [1 7 2]] 

[[4 3 6]
 [0 8 5]
 [1 7 2]] 

[[4 3 6]
 [8 0 5]
 [1 7 2]] 

[[4 3 6]
 [8 7 5]
 [1 0 2]] 



True